In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import pipeline
import csv

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from scipy.sparse import hstack
import sys
sys.path.append('/Users/phyan/Desktop/Baseline/Identify Names /')
from helpe import *
from importlib import reload
reload(helpe)

from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

<module 'helpe' from '/Users/phyan/Desktop/Baseline/Identify Names /helpe.py'>

In [ ]:
data = {
    'name': ['John Doe', 'Jane Smith', 'ACME Corp', 'Tech Innovations LLC', 'Maria Garcia'],
    'label': [0, 0, 1, 1, 0]  # 0: human name, 1: company name
} # for training 
df = pd.DataFrame(data)

X_train, X_test, y_train, y_test = train_test_split(df['name'], df['label'], test_size=0.2, random_state=42)



vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)


Below is for manually doing this 
df is the data of all lender.csv

In [19]:
df = pd.read_csv("lender_list.csv", usecols=["Lender"])
df.dropna(subset=['Lender'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("lender.csv",header=True, columns=df, index=False)
df = df[:2000]

In [21]:
df = parse(df)
df['target'] = df.apply(target, axis=1)

# df.to_csv("lender.csv",header=True, columns=df, index=False)

filtered_df_1 = df[df["target"] == 1]
filtered_df = df[df["target"] == 0]
filtered_df_2 = df[df["target"] == 2]

# Write the filtered DataFrame to a new CSV file
# filtered_df.to_csv('output_filtered.csv', index=False)
# filtered_df_1.to_csv("lender_company_grantee.csv", index=False)

filtered_df_1.to_csv("lender_company_grantee.csv",header=True, index=False)
filtered_df.to_csv("lender_name.csv",header=True, index=False)
filtered_df_2.to_csv("lender_none.csv",header=True, index=False)


In [23]:
# combine the csv files 
df1 = pd.read_csv('lender_company_grantee.csv', header=None)
df2 = pd.read_csv('lender_name.csv', header=None)
df3 = pd.read_csv('lender_none.csv', header=None)

# Concatenate DataFrames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df.columns = ["Lender","two_words","contains_keyword","char_count","digit_count","special_char_count","uppercase_word_count","title_case_word_count","avg_word_length","contains_single_letter","target"]


# Write to a new CSV file
combined_df.to_csv('lender.csv', index=False, header=True)
df = pd.read_csv("lender.csv")


In [32]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Transform the text data to feature vectors
X_tfidf = vectorizer.fit_transform(df['Lender'])
df = parse(df)
# Combine all features
X = hstack([X_tfidf, additional(df)])
y = df['target']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
X_train_vec = vectorizer.fit_transform(list(X_train))
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression(multi_class='ovr')
model.fit(X_train_vec, y_train)


[<1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>, <1x2243 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Com

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [186]:
y_pred = model.predict(X_test_vec)

# Evaluate the model
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

           0       1.00      1.00      1.00       459
           1       1.00      1.00      1.00       811

    accuracy                           1.00      1270
   macro avg       1.00      1.00      1.00      1270
weighted avg       1.00      1.00      1.00      1270

Accuracy: 1.0


In [158]:
file_name = "lender_list.csv"

df = pd.read_csv(file_name, usecols=["Lender"])
df.dropna(subset=['Lender'], inplace=True)
df.reset_index(drop=True, inplace=True)
new_data = df['Lender'].tolist()

In [190]:

import joblib
import pandas as pd
from scipy.sparse import hstack

# Load the trained model and vectorizer
model = joblib.load('logistic_regression_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Preprocess the new data
new_data_tfidf = vectorizer.transform(new_data)
new_data = parse(new_data)

# Extract additional features
new_data_features = df[['two_words', 'contains_keyword', 'char_count', 'digit_count', 
                          'special_char_count', 'uppercase_word_count', 'title_case_word_count', 
                          'avg_word_length', 'contains_single_letter']].values

new_data_combined = hstack([new_data_tfidf, new_data_features])

# Combine all features
new_data_combined = hstack([new_data_tfidf, new_data_features])

# Make predictions
predictions = model.predict(new_data_combined)
predictions = list(predictions)

# Print predictions
# print(predictions) # Output: [1 0] means "New Tech Solutions Inc." is classified as an organization, "Jane Doe" as a name

filename = "Organization.csv"

# Write the list to the CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(predictions)):
        if predictions[i] == 1:
            writer.writerow([new_data[i]])

ValueError: X has 1749 features, but LogisticRegression is expecting 1748 features as input.